In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_csv('./Data_from2000.csv')
data.columns

Index(['FutCode', 'Date_', 'Open_', 'High', 'Low', 'Volume', 'Settlement',
       'OpenInterest', 'P', 'PB', 'PA', 'WA', 'CA', 'YD', 'HM', 'HA', 'MT',
       'LA', 'LM', 'OA', 'UP', 'ContrCode', 'ClsCode', 'DSMnem', 'LDB',
       'ContrDate', 'ContrDateFmt', 'ISOCurrCode', 'CurrUnitCode', 'UnitCode',
       'TrdStatCode', 'StartDate', 'LastTrdDate', 'SttlmntDate',
       'ExpirationDate', 'FirstNoticeDate', 'LastNoticeDate',
       'FirstDelvryDate', 'TickSizeUnitCode'],
      dtype='object')

In [3]:
##### Uncomment this to read from raw file and save as pickle #####
data = pd.read_csv(
    './Data_from2000.csv', 
    usecols=['FutCode','Date_','Open_', 'High', 'Low','Volume','CA','UP','DSMnem',
            'Settlement','OpenInterest','P','ContrCode','TrdStatCode',
            'StartDate', 'LastTrdDate', 'SttlmntDate','ExpirationDate'],
    dtype={'FutCode':int, 'Date_':str, 'Volume':float, 'P':float, 'CA':float, 'UP':float,'ContrCode':str,'DSMnem':str,'Open_':float, 'High':float, 'Low':float,
           'TrdStatCode':str,'StartDate':str,'LastTrdDate':str, 'SttlmntDate':str,'ExpirationDate':str})
data.to_pickle('./Data_from2000.pkl')

In [3]:
data = pd.read_pickle('./Data_from2000.pkl')

In [4]:
data

,FutCode,Date_,Open_,High,Low,Volume,Settlement,OpenInterest,P,CA,UP,ContrCode,DSMnem,TrdStatCode,StartDate,LastTrdDate,SttlmntDate,ExpirationDate
0,1,20140404,4.610000,4.610000,4.610000,0.0,4.610000,645.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
1,1,20150513,3.650999,3.650999,3.650999,0.0,3.650999,893.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
2,1,20170727,3.035999,3.035999,3.035999,48.0,3.044999,11081.0,2.0,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
3,1,20160516,3.074000,3.074000,3.074000,6.0,3.074000,1147.0,NaN,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
4,1,20180920,3.006000,3.006000,3.006000,3372.0,3.030999,34655.0,12.0,NaN,NaN,3888,NHD1218,D,20121128,20181128,21517,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27470602,443657,20221129,1907.500000,1907.500000,1907.500000,0.0,1907.500000,0.0,NaN,NaN,NaN,3223,NMG1024,A,20221128,20241029,23678,NaN
27470603,443658,20221129,3.618000,3.618000,3.618000,0.0,3.618000,0.0,NaN,NaN,NaN,2083,NQC1124,A,20221128,20241029,23678,NaN
27470604,443659,20221129,2.817000,2.817000,2.817000,0.0,2.817000,NaN,NaN,NaN,NaN,2085,NQH1223,A,20221129,20231129,23343,NaN
27470605,443660,20221129,2.180000,2.180000,2.180000,0.0,2.180000,NaN,NaN,NaN,NaN,2090,NQU1223,A,20221129,20231129,23343,NaN


In [5]:
example_dictionary = {
        'Copper':'NHG0100',
        'Cotton':'NCT0319',
        'Cocoa':'NCC0919',
        'Corn':'CC.0503',
        'Wheat':'CW.0301',
        'Soybeans':'CS.0100',
        'Soy oil':'CBO0101',
        'Soy meat':'CMS0100',
        'Oats':'CO.0300',
        'Sugar':'NSB0103',
        'Pork bellies':'CPB0212',
        'Silver':'NSL0100',
        'Live cattle':'CLC0410',
        'Lean hogs':'CLH0200',
        'Orange juice':'NJO0100',
        'Platinum':'NPL0100',
        'Lumber':'ILB0109',
        'Feeder cattle':'CFC0100',
        'Coffee':'NKC0300',
        'Gold':'NGC0105',
        'Palladium':'NPA0112',
        'Heat oil':'NHO0103',
        'Crude oil':'NCL0609',
        'Gasoline unleaded':'NHU0400',
        'Gasoline blendstock':'NRB0610',
        'Rough rice':'CNR0102',
        'Natural gas':'NNG0119',
        'Milk':'CCM0102',
        'Coal':'NQL0614',
        'Butter':'CCB1219'
    }

In [6]:
def find_contracode_with_commodity_name(data, com_name, example_dictionary):
    """
    find contract code with given commodity_name.
    A contract code is corresponding to several contracts with different expiry date.
    """
    DSMnem = data['DSMnem'].values
    com_name_code = example_dictionary[com_name]
    if com_name_code in DSMnem:
        if len(data[data['DSMnem']==com_name_code]['ContrCode'].unique())!=1:
            print('--------------------')
            print(data[data['DSMnem']==com_name_code]['ContrCode'].unique())
            print('logic not correct')
            print('--------------------')
        return data[data['DSMnem']==com_name_code]['ContrCode'].iloc[0]
    else:
        print('None exist')


In [7]:
def extract_and_label_data(data, example_dictionary):
    # find contract code to commodity relations
    contract_code_to_commodity = {}
    print('find commodity name to contract code relation')
    for key in tqdm(example_dictionary):
        _code = find_contracode_with_commodity_name(data, key, example_dictionary)
        contract_code_to_commodity[_code] = key
    # extract selected commodity data and label it
    temp_data_lst = []
    print('extract and label data')
    for _code in tqdm(contract_code_to_commodity.keys()):
        _temp_data = data[data['ContrCode']==_code]
        _temp_data['Commodity_Type'] = contract_code_to_commodity[_code]
        temp_data_lst.append(_temp_data)
    data_selected = pd.concat(temp_data_lst, axis=0, ignore_index=True)
    return data_selected

In [8]:
data.columns

Index(['FutCode', 'Date_', 'Open_', 'High', 'Low', 'Volume', 'Settlement',
       'OpenInterest', 'P', 'CA', 'UP', 'ContrCode', 'DSMnem', 'TrdStatCode',
       'StartDate', 'LastTrdDate', 'SttlmntDate', 'ExpirationDate'],
      dtype='object')

In [9]:
data_selected = extract_and_label_data(data, example_dictionary)

find commodity name to contract code relation


100%|██████████| 30/30 [01:30<00:00,  3.03s/it]


extract and label data


100%|██████████| 30/30 [00:38<00:00,  1.28s/it]


In [10]:
data_selected.to_pickle('./data_selected_from2000.pkl')

In [11]:
copper = data_selected[data_selected['Commodity_Type']=='Copper']

In [12]:
copper.sort_values(['Date_','LastTrdDate']).head(100)

,FutCode,Date_,Open_,High,Low,Volume,Settlement,OpenInterest,P,CA,UP,ContrCode,DSMnem,TrdStatCode,StartDate,LastTrdDate,SttlmntDate,ExpirationDate,Commodity_Type
17,494,20000103,0.8525,0.8525,0.8525,481.0,0.8525,2620.0,NaN,NaN,NaN,2026,NHG0100,D,19980130,20000127,14641,NaN,Copper
32974,83294,20000103,0.8550,0.8580,0.8520,213.0,0.8580,3119.0,NaN,NaN,NaN,2026,NHG0200,D,19980227,20000225,14669,NaN,Copper
62,1557,20000103,0.8600,0.8640,0.8555,4860.0,0.8630,45034.0,NaN,NaN,NaN,2026,NHG0300,D,19980514,20000329,14701,NaN,Copper
4345,12570,20000103,0.8630,0.8655,0.8630,32.0,0.8660,1150.0,NaN,NaN,NaN,2026,NHG0400,D,19980601,20000426,14729,NaN,Copper
42014,115563,20000103,0.8670,0.8700,0.8625,101.0,0.8690,5339.0,NaN,NaN,NaN,2026,NHG0500,D,19980624,20000526,14759,NaN,Copper
18924,49371,20000103,0.8710,0.8710,0.8710,7.0,0.8710,1086.0,NaN,NaN,NaN,2026,NHG0600,D,19980630,20000628,14792,NaN,Copper
43726,126246,20000103,0.8705,0.8740,0.8690,65.0,0.8735,4383.0,NaN,NaN,NaN,2026,NHG0700,D,19980803,20000727,14823,NaN,Copper
84626,175833,20000103,0.8760,0.8760,0.8760,0.0,0.8760,811.0,NaN,NaN,NaN,2026,NHG0800,D,19980914,20000829,14854,NaN,Copper
14545,29886,20000103,0.8790,0.8790,0.8790,20.0,0.8790,3731.0,NaN,NaN,NaN,2026,NHG0900,D,19980930,20000927,14883,NaN,Copper
7490,17845,20000103,0.8805,0.8805,0.8805,0.0,0.8805,544.0,NaN,NaN,NaN,2026,NHG1000,D,19981104,20001027,14914,NaN,Copper
